In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from urllib.parse import urlsplit
import pandas as pd
import requests
import re
import time

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [4]:
browser.visit(url)

In [5]:
results = browser.html
soup = BeautifulSoup(results, 'html.parser')

In [6]:
results_raw = soup.find("div", class_="list_text")
results_raw

<div class="list_text"><div class="list_date">November 16, 2018</div><div class="content_title"><a href="/news/8386/nasa-to-host-media-call-on-next-mars-landing-site/" target="_self">NASA to Host Media Call on Next Mars Landing Site</a></div><div class="article_teaser_body">NASA will host a media teleconference at 9 a.m. PST (noon EST) Monday, Nov. 19, to provide details about the Mars 2020 rover’s landing site on the Red Planet.</div></div>

In [7]:
results_1 = soup.find("div", class_="content_title").text

In [8]:
print(results_1)

NASA to Host Media Call on Next Mars Landing Site


In [9]:
results_2 = soup.find("div", class_="article_teaser_body").text

In [10]:
print(results_2)

NASA will host a media teleconference at 9 a.m. PST (noon EST) Monday, Nov. 19, to provide details about the Mars 2020 rover’s landing site on the Red Planet.


In [11]:
image_mining = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_mining)

In [12]:
#Finding our base URL
image_ore_1 = "{0.scheme}://{0.netloc}".format(urlsplit(image_mining))
print(image_ore_1)

https://www.jpl.nasa.gov


In [13]:
click_path = "//*[@id=\"page\"]/section[3]/div/ul/li[1]/a/div/div[2]/img"
results_3 = browser.find_by_xpath(click_path)
img = results_3[0]
img.click()
time.sleep(5)

In [14]:
featured_image = browser.html

In [15]:
soup2 = BeautifulSoup(featured_image, "html.parser")
image_ore_2 = soup2.find("img", class_="fancybox-image")

string_image = str(image_ore_2)
string_image

'<img class="fancybox-image" src="/spaceimages/images/largesize/PIA22849_hires.jpg" style="display: inline;"/>'

In [16]:
#Some regex. Its a long story...
string_image_2 = re.findall('{}(.*){}'.format('src="', '" style'), string_image)
string_image_2[0]

'/spaceimages/images/largesize/PIA22849_hires.jpg'

In [17]:
featured_img_url = image_ore_1 + string_image_2[0]
print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA22849_hires.jpg


In [18]:
#Martian Weather Twitter

martian_weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(martian_weather_url)

In [19]:
martian_weather = browser.html
soup3 = BeautifulSoup(martian_weather, "html.parser")

In [20]:
temperature = soup3.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})
mars_weather = soup3.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
print(mars_weather)

Sol 2230 (2018-11-14), high -5C/23F, low -72C/-97F, pressure at 8.59 hPa, daylight 06:22-18:39


In [21]:
mars_facts = "https://space-facts.com/mars/"

In [22]:
fact_sheet = pd.read_html(mars_facts)
fact_sheet[0]

0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.52 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                  -153 to 20 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers

In [23]:
fact_sheet_df = fact_sheet[0]
fact_sheet_df.columns = ["Measurement Type", "Measurement"]
fact_sheet_df.set_index(["Measurement Type"])

Measurement
Measurement Type                                   
Equatorial Diameter:                       6,792 km
Polar Diameter:                            6,752 km
Mass:                 6.42 x 10^23 kg (10.7% Earth)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.52 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                  -153 to 20 °C
First Record:                     2nd millennium BC
Recorded By:                   Egyptian astronomers

In [24]:
mars_table = fact_sheet_df.to_html()
mars_table = mars_table.replace("\n", "")
mars_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Measurement Type</th>      <th>Measurement</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egypt

In [25]:
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)

In [26]:
base_hemi = "{0.scheme}://{0.netloc}/".format(urlsplit(hemi_url))
print(base_hemi)

https://astrogeology.usgs.gov/


In [27]:
hemi_urls = []
results_4 = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div[1]/a/img").click()
time.sleep(2)
open_cerberus = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
time.sleep(1)
image_cerberus = browser.html
soup = BeautifulSoup(image_cerberus, "html.parser")
url_cerberus = soup.find("img", class_="wide-image")["src"]

image_url_cerberus = base_hemi + url_cerberus
print(image_url_cerberus)

title_cerberus = soup.find("h2",class_="title").text
print(title_cerberus)

#Back and save
press_back = browser.find_by_xpath("//*[@id='splashy']/div[1]/div[1]/div[3]/section/a").click()
cerberus = {"image title":title_cerberus, "image url": image_url_cerberus}
hemi_urls.append(cerberus)

https://astrogeology.usgs.gov//cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
Cerberus Hemisphere Enhanced


In [28]:
results_5 = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div[2]/a/img").click()
time.sleep(2)
open_schiaparelli = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
time.sleep(1)
image_schiaparelli = browser.html
soup = BeautifulSoup(image_schiaparelli, "html.parser")
url_schiaparelli = soup.find("img", class_="wide-image")["src"]

image_url_schiaparelli = base_hemi + url_schiaparelli
print(image_url_schiaparelli)

title_schiaparelli = soup.find("h2",class_="title").text
print(title_schiaparelli)

#Back and save
press_back_2 = browser.find_by_xpath("//*[@id='splashy']/div[1]/div[1]/div[3]/section/a").click()
schiaparelli = {"image title":title_schiaparelli, "image url": image_url_schiaparelli}
hemi_urls.append(schiaparelli)

https://astrogeology.usgs.gov//cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
Schiaparelli Hemisphere Enhanced


In [29]:
#In case you are wondering why am repeating this from earlier: the clicking macro is a tad touchy.
#Its working more reliably split up into two parts.

hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)

base_hemi = "{0.scheme}://{0.netloc}/".format(urlsplit(hemi_url))
print(base_hemi)

https://astrogeology.usgs.gov/


In [30]:
results_6 = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div[3]/a/img").click()
time.sleep(2)
open_syrtis_major = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
time.sleep(1)
image_syrtis_major = browser.html
soup = BeautifulSoup(image_syrtis_major, "html.parser")
url_syrtis_major = soup.find("img", class_="wide-image")["src"]

image_url_syrtis_major = base_hemi + url_syrtis_major
print(image_url_syrtis_major)

title_syrtis_major = soup.find("h2",class_="title").text
print(title_syrtis_major)

#Back and save
press_back_3 = browser.find_by_xpath("//*[@id='splashy']/div[1]/div[1]/div[3]/section/a").click()
syrtis_major = {"image title": title_syrtis_major, "image url": image_url_syrtis_major}
hemi_urls.append(syrtis_major)

https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg
Syrtis Major Hemisphere Enhanced


In [31]:
results_7 = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div[4]/a/img").click()
time.sleep(2)
open_valles_marineris = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
time.sleep(1)
image_valles_marineris = browser.html
soup = BeautifulSoup(image_valles_marineris, "html.parser")
url_valles_marineris = soup.find("img", class_="wide-image")["src"]

image_url_valles_marineris = base_hemi + url_valles_marineris
print(image_url_valles_marineris)

title_valles_marineris = soup.find("h2",class_="title").text
print(title_valles_marineris)

#Back and save
press_back_4 = browser.find_by_xpath("//*[@id='splashy']/div[1]/div[1]/div[3]/section/a").click()
valles_marineris = {"image title": title_valles_marineris, "image url": image_url_valles_marineris}
hemi_urls.append(valles_marineris)

https://astrogeology.usgs.gov//cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg
Valles Marineris Hemisphere Enhanced


In [32]:
print(hemi_urls)

[{'image title': 'Cerberus Hemisphere Enhanced', 'image url': 'https://astrogeology.usgs.gov//cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'image title': 'Schiaparelli Hemisphere Enhanced', 'image url': 'https://astrogeology.usgs.gov//cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'image title': 'Syrtis Major Hemisphere Enhanced', 'image url': 'https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'image title': 'Valles Marineris Hemisphere Enhanced', 'image url': 'https://astrogeology.usgs.gov//cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
